In [1]:
import minsearch
import json

# Retriever Configuration

In [2]:
with open('documents.json') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Minsearch

In [5]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)
index.fit(documents)

In [6]:
q = "how much time does the course take?"

boost = {
    'course': 1.0,
    'question': 3.0,
    'text': 1.0,
    'section': 0.5
}
filter_dict = {
    'course':'data-engineering-zoomcamp'
}
index.search(
    query=q,
    boost_dict=boost,
    # filter_dict=filter_dict,
    )

[{'text': 'Around ~10 hours per week. Timur Kamaliev did a detailed analysis of how much time students of the previous cohort needed to spend on different modules and projects. Full article',
  'section': 'General course-related questions',
  'question': 'How much time do I need for this course?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.\nYou can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.',
  'section': 'General course-related questions',
  'question': 'Course - Can I get support if I take the course in the self-paced mode?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes! We'll cover some linear algebra in the course, but in general, there will be v

## Elasticsearch

In [7]:
from elasticsearch import Elasticsearch
es_client= Elasticsearch('http://localhost:9200')

In [8]:
es_client.info()

ObjectApiResponse({'name': '9611ea0df813', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'U7-onk54REO4et3LmXCnCg', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name,
                document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
def elastic_search(query):
        
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)
    return result_docs 


In [13]:
query = "I just discovered the course! can I still join it?"
results = elastic_search(query)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

# Generator Configuration

In [14]:
from openai import OpenAI
client= OpenAI()

In [30]:
prompt_template = """
You are a helpful assistant for answering questions about various courses.
When you get a question, use the provided CONTEXT to answer it.
Use only the facts from the CONTEXT to answer the question.

QUESTION: {question}

CONTEXT: {context}
"""

In [59]:
q = "what prerequisites are needed for the course?"

In [60]:
results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict=filter_dict,
)

In [61]:
results[1]

{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
 'section': 'General course-related questions',
 'question': 'Course - What can I do before the course starts?',
 'course': 'data-engineering-zoomcamp'}

In [62]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nresponse: {doc['text']}\n\n"

In [63]:
prompt = prompt_template.format(
    question=q,
    context=context
)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}
    ],
)
    

In [65]:
response

ChatCompletion(id='chatcmpl-BdbnwMe4Vwe3tS9o8x5Ngh891PvpO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The prerequisites for the course can be found here: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1748779812, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_a288987b44', usage=CompletionUsage(completion_tokens=28, prompt_tokens=1173, total_tokens=1201, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [66]:
print(response.choices[0].message.content)

The prerequisites for the course can be found here: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites.


# Modularization

In [15]:
def search(query):
    
    boost = {
        'course': 1.0,
        'question': 3.0,
        'text': 1.0,
        'section': 0.5
    }

    filter_dict = {
        'course':'data-engineering-zoomcamp'
    }

    results = index.search(
        query=query,
        boost_dict=boost,
        # filter_dict=filter_dict,
        num_results=3
        )
    
    return results

    

In [35]:
def elastic_search(query):
        
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
            #     "filter": {
            #         "term": {
            #             "course": "data-engineering-zoomcamp"
            #         }
            #     }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)
    return result_docs 


In [16]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a helpful assistant for answering questions about various courses.
    When you get a question, use the provided CONTEXT to answer it.
    Use only the facts from the CONTEXT to answer the question.

    QUESTION: {question}

    CONTEXT: {context}
    """
    prompt = prompt_template.format(
        question=query,
        context=search_results
    )
    return prompt

In [17]:
from openai import OpenAI
client= OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message.content
    

In [18]:
def rag(query):
    # search_results = search(query)
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [20]:
query = "I just discovered the course! can I still join it?"
response = rag(query)
print(response)

Yes, you can still join the course even after the start date. You don't need to register to participate, and you can submit the homework assignments. However, be mindful of the deadlines for turning in the final projects, so make sure not to leave everything until the last minute.


# Homework

#### 1.

In [23]:
!curl -X GET "http://localhost:9200"

{
  "name" : "9611ea0df813",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "U7-onk54REO4et3LmXCnCg",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


The `"build_hash"` value is `"dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"`

#### 2.

In [21]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name,
                document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

#### 3.

In [ ]:
def elastic_search(query):
        
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            #     "filter": {
            #         "term": {
            #             "course": "data-engineering-zoomcamp"
            #         }
            #     }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)
    return result_docs 


In [30]:
query= "How do execute a command on a Kubernetes pod?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            #     "filter": {
            #         "term": {
            #             "course": "data-engineering-zoomcamp"
            #         }
            #     }
            }
        }
    }
response = es_client.search(index=index_name, body=search_query)

In [32]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'xqi2OpcB7k5QMZnUMDIB',
  '_score': 44.55304,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'eqi6OpcB7k5QMZnUHjam',
  '_score': 44.55304,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> 

The max document score is 44.55304.

#### 4.

In [36]:
query = "How do copy a file to a Docker container?"

search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': '5ai2OpcB7k5QMZnUMDJF',
 '_score': 66.94253,
 '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
  'course': 'machine-learning-zoomcamp'}}

#### 5.

In [44]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


def build_context_prompt(docs_raw):
    context = ""
    for doc in docs_raw:
        context += context_template.format(
            question=doc['question'],
            text=doc['text']
        ) + "\n\n"
    return context

def build_prompt(query, context_docs):
    return prompt_template.format(
        question=query,
        context=build_context_prompt(context_docs)
    )


In [45]:
def search(query):
    search_query = {
            "size": 3,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    return [x['_source'] for x in response['hits']['hits']]

In [46]:
query = "How do I execute a command in a running docker container?"
response = search(query)
prompt = build_prompt(query, response)
print(len(prompt))
print(prompt)

1401
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my 

#### 6.

In [51]:
import tiktoken

In [52]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [53]:
len(encoding.encode(prompt))

311

#### Bonus 1.

In [55]:
from openai import OpenAI
client= OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message.content
print(llm(prompt))

To execute a command in a running Docker container, you can use the `docker exec` command. First, identify the container ID by using the `docker ps` command. Then, execute your desired command inside the container using:

```
docker exec -it <container-id> <command>
```

For example, if you want to start a bash session in a running container, you would use:

```
docker exec -it <container-id> bash
```


#### Bonus 2.

__GPT-4o__
- $5.00 / 1M input tokens
- $2.50 / 1M cached input tokens
- $20.00 / 1M output tokens

In [58]:
input_count = 150
output_count = 250

input_cost = input_count/1000000 * 5
output_cost = output_count/1000000 * 20
total_cost = input_cost + output_cost

n_requests = 1000

print(f"Total cost for {n_requests} requests: ${total_cost * n_requests:.2f}")

Total cost for 1000 requests: $5.75
